In [2]:
import os
import sys
import json
import torch
from tqdm import tqdm
from typing import List
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from peft import PeftConfig, PeftModel

## Загрузка и сохранение на диск базовой модели.

In [ ]:
# MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
# template_path = '/home/dshome/Projects/rulm/self_instruct/internal_prompts/saiga_v2.json'
# config = PeftConfig.from_pretrained(MODEL_NAME)

# base_model_config = AutoConfig.from_pretrained(config.base_model_name_or_path)

# base_model = AutoModelForCausalLM.from_pretrained(
#     config.base_model_name_or_path,
#     torch_dtype=base_model_config.torch_dtype,
#     load_in_8bit=True,
#     device_map="auto",
# )
# base_model.save_pretrained('models/base_model/')

In [ ]:
# Загружаем базовую модель. 
# Загружаем Адаптер.
# Загружаем токенайзер.

## Тестирование работы адаптера.

In [3]:
base_model_path = 'models/base_model/'
adapter_path = 'models/'
tockenizer_path = 'models/'

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(tockenizer_path, use_fast=False) #, padding_side='left')
# generation_config = GenerationConfig.from_pretrained(model_name, do_sample=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
quantization_config=BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        # bnb_4bit_compute_dtype=self.torch_dtype,
        # bnb_4bit_use_double_quant=True,
        # bnb_4bit_quant_type="nf4"
    ),

In [6]:
generation_config = GenerationConfig.from_pretrained(base_model_path,
                                                     do_sample=True)

In [8]:
# config = PeftConfig.from_pretrained(adapter_path)
# base_model_config = AutoConfig.from_pretrained(base_model_path)

# torch_dtype = base_model_config.torch_dtype

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    load_in_8bit=True,
    quantization_config=quantization_config,
    # torch_dtype=torch_dtype,
    device_map="auto",
)

AttributeError: 'tuple' object has no attribute 'load_in_8bit'

In [ ]:
model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    adapter_name="adapter1_name",
    # torch_dtype=torch_dtype
)

In [ ]:
model.eval()

In [ ]:
print(model.active_adapter)

adapter1_name


In [ ]:
# Выполнить предсказание на модели.

# Посмотреть скрипты для загрузки модели и всех данных со своего google-drive.
# Надо еще посомотереть как зделать chackpoint на свой google-drive. 
# Чтобы не терять данные модели.
# Это в случае, если модель не влезал на мою видеокарту.  

In [ ]:
system_prompt = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

In [ ]:
user_message =  "Как приготовить борщ?"

In [ ]:
s = f"<s>system\n{system_prompt}</s>\n" + \
    f"<s>user\n{user_message}</s>\n" + \
    f"<s>bot\n"

In [ ]:
data = tokenizer(s,return_tensors="pt")

In [ ]:
data = {k: v.to(device) for k, v in data.items()}

In [ ]:
# Настроить параметры генерации текста.
output_ids = model.generate(**data, do_sample=True, temperature=0.9, max_new_tokens=512, top_p=0.95,
                             top_k=5, repetition_penalty=1.03, no_repeat_ngram_size=2, generation_config=generation_config)

In [ ]:
# Переписать.
outputs = []
for sample_output_ids, sample_input_ids in zip(output_ids, data["input_ids"]):
    sample_output_ids = sample_output_ids[len(sample_input_ids):]
    sample_output = tokenizer.decode(sample_output_ids, skip_special_tokens=True)
    sample_output = sample_output.replace("</s>", "").strip()
    outputs.append(sample_output)
outputs[0]

'Борщ — национальный украинский суп, который можно приготовлять различными способами в зависимости от местных традиций или вкусовых предпочтений. Вот несколько популярных рецептов:\n\n1. Основной реcipe: нарежьте картофель, морковь, лук, чеснок и кольца помидоров, затем обжарь их на оливковом масле. Добавьte воду, уксус и бульон, и варите в течение 20-30 минут до готовности. В конце добавляйте сливочное масло, зеленый луковичий перец и зелени (укроп или петрушку), и подавай.\n2. Украинская борщу: используй для начинки крутокоренники, курицу, говядину, конину или баранины, которые нужно нашинковать, обжарить на сковороде и дополнительно обсушить. Затем, следует применять основной прием, но с использованием мяса вместо картин.  Это более сложный и насыщенный реципи.  \n3. Борщу из грибов: сначала обжимай глубокий миска грыбной массы, добиваясь ее настойчивой текстуры. Потом следуют основные ингредиенты, а гренки можно додавать в конце, чтобы не испортить текст. Этот вариант является боле

Борщ — национальный украинский суп, который можно приготовлять различными способами в зависимости от местных традиций или вкусовых предпочтений. Вот несколько популярных рецептов:\n\n1. Основной реcipe: нарежьте картофель, морковь, лук, чеснок и кольца помидоров, затем обжарь их на оливковом масле. Добавьte воду, уксус и бульон, и варите в течение 20-30 минут до готовности. В конце добавляйте сливочное масло, зеленый луковичий перец и зелени (укроп или петрушку), и подавай.\n2. Украинская борщу: используй для начинки крутокоренники, курицу, говядину, конину или баранины, которые нужно нашинковать, обжарить на сковороде и дополнительно обсушить. Затем, следует применять основной прием, но с использованием мяса вместо картин.  Это более сложный и насыщенный реципи.  \n3. Борщу из грибов: сначала обжимай глубокий миска грыбной массы, добиваясь ее настойчивой текстуры. Потом следуют основные ингредиенты, а гренки можно додавать в конце, чтобы не испортить текст. Этот вариант является более легким и менее пикантным.   \n4. Соргоборщ: в качестве основы используется сорг, которое можно купить в магазинах или подготавить самостоятельно, смешав кукурузное зерно с мукой. Оно дает борщину более сладкую, менестреличную текстовую структуру, что делает ей более мягкой и нежной.    \n5. Супер-легкая борш: для этого варенье используют карта